## Install dependencies and imports

In [ ]:
!pip install transformers torch torchvision Pillow roboflow
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

## Load FaceBook's DETR model

In [ ]:
# Load pre-trained DETR model and processor
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

## Import dataset of OSHA/ANSI violations

In [ ]:
# Creating datasets directory
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# Get dataset from Roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="DVSACmCnjnq3Yb7MimiW")
project = rf.workspace("vit16").project("hard-hat-sample-kc3iw")
dataset = project.version(4).download("coco")

# Check where dataset is located
dataset.location

In [ ]:
import os
import torchvision

_annotations = "_annotations.coco.json"
train_folder = os.path.join(dataset.location, "train")
val_folder = os.path.join(dataset.location, "valid")
test_folder = os.path.join(dataset.location, "test")


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        image_directory_path: str,
        processor,
        train: bool = True
    ):
        annotation_file_path = os.path.join(image_directory_path, _annotations)
        super(CocoDetection, self).__init__(image_directory_path, annotation_file_path)
        self.processor = processor

    def __getitem__(self, idx):
        images, annotations = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': annotations}
        encoding = self.processor(images=images, annotations=annotations, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]

        return pixel_values, target


train_dataset = CocoDetection(image_directory_path=train_folder, processor=processor, train=True)
val_dataset = CocoDetection(image_directory_path=val_folder, processor=processor, train=False)
test_dataset = CocoDetection(image_directory_path=test_folder, processor=processor, train=False)

print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(val_dataset))
print("Number of test examples:", len(test_dataset))

## Train the Model